# Business Understanding

## Project Overview
For this project, you will use exploratory data analysis to generate insights for a business stakeholder.

## Business problem: 
Your company now sees all the big companies creating original video content and they want to get in on the fun. They have decided to create a new movie studio, but they don’t know anything about creating movies. You are charged with exploring what types of films are currently doing the best at the box office. You must then translate those findings into actionable insights that the head of your company's new movie studio can use to help decide what type of films to create.
## Project objectives: 
### Main Objective
To analyze movie data and uncover patterns in sales, popularity, ratings, and director influence across genres, providing actionable insights for business growth and strategy.

### Specific Objectives
1. **Genre by Sales**  
   Identify which genres generate the most revenue and analyze trends contributing to their sales performance.
   Tables:`bom.movie_gross.csv` , `rt.movie_info.tsv`

2. **Genre by Popularity**  
   Understand which genres are most popular among audiences and explore factors driving their popularity.
   Tables:`bom.movie_gross.csv`, `tmdb.movies.csv`

3. **Genre by Rating**  
   Examine the ratings of movies across different genres to evaluate their critical reception.
    DB:`im.db` Tables: `movie basics`, `movie rating`

4. **Directors by Genre**  
   Determine which directors are most associated with specific genres and assess their impact on genre success.
    DB:`im.db` Tables: `movie_basics`, `directors`

### The Data
In the folder `zippedData` are movie datasets from:

* [Box Office Mojo](https://www.boxofficemojo.com/)
* [IMDB](https://www.imdb.com/)
* [Rotten Tomatoes](https://www.rottentomatoes.com/)
* [TheMovieDB](https://www.themoviedb.org/)
* [The Numbers](https://www.the-numbers.com/)

Because it was collected from various locations, the different files have different formats. Some are compressed CSV (comma-separated values) or TSV (tab-separated values) files that can be opened using spreadsheet software or `pd.read_csv`, while the data from IMDB is located in a SQLite database.

![movie data erd](https://raw.githubusercontent.com/learn-co-curriculum/dsc-phase-2-project-v3/main/movie_data_erd.jpeg)

Note that the above diagram shows ONLY the IMDB data. You will need to look carefully at the features to figure out how the IMDB data relates to the other provided data files.

It is up to you to decide what data from this to use and how to use it. If you want to make this more challenging, you can scrape websites or make API calls to get additional data. If you are feeling overwhelmed or behind, we recommend you use only the following data files:

* `im.db.zip`
  * Zipped SQLite database (you will need to unzip then query using SQLite)
  * `movie_basics` and `movie_ratings` tables are most relevant
* `bom.movie_gross.csv.gz`
  * Compressed CSV file (you can open without expanding the file using `pd.read_csv`)

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

# Data Understanding 

# Data Preparation


# Modeling


# Evaluation